In [1]:
import numpy as np 
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("dataset/ask_play_station.csv")
df.dropna(inplace = True)
df

,User,Company
0,"@AskPlayStation So, what's the november ps plu...",@115743 There is no info to share at the momen...
1,@AskPlayStation It was when I would go to down...,"@115745 Glad to know that is downloading, ple..."
2,@AskPlayStation I bought Lego Star Wars in the...,@115745 Sorry for the inconvenience. Do you se...
3,@AskPlayStation can i block a community from s...,@115746 Glad to help. Please share the detai...
4,@AskPlayStation can you dm me I have a question,@116001 Glad to help! We have sent you a DM lo...
...,...,...
16711,@AskPlayStation I have already read and went o...,@640122 Please run a network connection test. ...
16712,@AskPlayStation I can not fully connect to my ...,@640122 Let's take a look! Check out the next ...
16713,@AskPlayStation i want to buy a ps4 pro . will...,@823562 Hello there. There isn't information a...
16714,"@AskPlayStation any idea when ""Steven Universe...",@823563 Glad to help! Please follow us via Tw...


In [177]:
df = pd.read_csv("dataset/amazon_data.csv")
df.dropna(inplace = True)
df = df[:15000]
df

,User,Company
0,Way to drop the ball on customer service @1158...,@115820 I'm sorry we've let you down! Without ...
1,@AmazonHelp 3 different people have given 3 di...,@115820 We'd like to take a further look into ...
2,@115823 I want my amazon payments account CLOS...,@115822 I am unable to affect your account via...
3,@115828 How about you guys figure out my Xbox ...,@115826 I'm sorry for the wait. You'll receive...
4,@AmazonHelp @115826 Yeah this is crazy we’re l...,@115827 Thanks for your patience. ^KM
...,...,...
15037,I'm so mad at @115821 rn.\n\nThe katana sword ...,@183848 I'm so sorry for the delay! We strive ...
15038,@AmazonHelp @183848 WORST SERVICE\nPLZ REPLY T...,@121400 We've responded to your DM as requeste...
15039,"@AmazonHelp Actually, email says Friday, but w...",@183848 Some items are not available to ship i...
15040,@AmazonHelp Why would I even be given an optio...,@183848 Our most accurate delivery date will b...


In [178]:
for index in df.index:
    df.loc[index,'Company'] = '<SOS> ' + df.loc[index,'Company'] + ' <EOS>'
df

,User,Company
0,Way to drop the ball on customer service @1158...,<SOS> @115820 I'm sorry we've let you down! Wi...
1,@AmazonHelp 3 different people have given 3 di...,<SOS> @115820 We'd like to take a further look...
2,@115823 I want my amazon payments account CLOS...,<SOS> @115822 I am unable to affect your accou...
3,@115828 How about you guys figure out my Xbox ...,<SOS> @115826 I'm sorry for the wait. You'll r...
4,@AmazonHelp @115826 Yeah this is crazy we’re l...,<SOS> @115827 Thanks for your patience. ^KM <EOS>
...,...,...
15037,I'm so mad at @115821 rn.\n\nThe katana sword ...,<SOS> @183848 I'm so sorry for the delay! We s...
15038,@AmazonHelp @183848 WORST SERVICE\nPLZ REPLY T...,<SOS> @121400 We've responded to your DM as re...
15039,"@AmazonHelp Actually, email says Friday, but w...",<SOS> @183848 Some items are not available to ...
15040,@AmazonHelp Why would I even be given an optio...,<SOS> @183848 Our most accurate delivery date ...


In [179]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

In [180]:
text_data = np.concatenate((train_data['User'].values, train_data['Company'].values))
text_data

array(['@115850 @115821 I am victim of fake product sold on your site (Order # 407-3795266-1584340)',
       '@AmazonHelp i need to speak with you urgently!',
       '@AmazonHelp But there are options like login and mail but after login how will I do something u had blocked my privileges',
       ...,
       '<SOS> @145378 Usually, order cancellation is accompanied by an email with the reason mentioned in it. Please check it here: https://t.co/NTkrxpsbHJ. ^SF <EOS>',
       '<SOS> @120613 Sorry for the inconvenience. You may check the email from our team here: https://t.co/NTkrxpsbHJ if you are unable (1/2) ^KA <EOS>',
       "<SOS> @155068 I understand your concern regarding the launch of kindle series. However, we haven't made any announcement on it yet. Please stay tuned. ^SD <EOS>"],
      dtype=object)

In [212]:
MAX_NB_WORDS = 7000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 22455 unique tokens.


In [213]:
word_counts  = tokenizer.word_counts 
rare_words_number = 0 
for index, (key, value) in enumerate(word_counts.items()):
    if (value < 2):
        rare_words_number = rare_words_number + 1
        #print( index, key, value)
rare_words_number

12695

In [214]:
train_data['User'].values[0]

'@115850 @115821 I am victim of fake product sold on your site (Order # 407-3795266-1584340)'

In [215]:
seq = tokenizer.texts_to_sequences([train_data['User'].values[0]])
seq

[[40, 62, 6, 106, 33, 582, 103, 528, 20, 9, 402, 27, 830]]

In [216]:
tokenizer.sequences_to_texts(seq)

['115850 115821 i am of fake product sold on your site order 407']

In [217]:
train_X = tokenizer.texts_to_sequences(train_data['User'].values)
train_y = tokenizer.texts_to_sequences(train_data['Company'].values)

In [218]:
train_X[0]

[40, 62, 6, 106, 33, 582, 103, 528, 20, 9, 402, 27, 830]

In [219]:
MAX_QUESTION_LENGTH = 50
#for el in df['User']:
 #   if(len(el) > MAX_QUESTION_LENGTH):
#        MAX_QUESTION_LENGTH = len(el)
print(MAX_QUESTION_LENGTH)

MAX_ANSWER_LENGTH = 50
#for el in df['Company']:
    #if(len(el) > MAX_ANSWER_LENGTH):
        #MAX_ANSWER_LENGTH = len(el)
print(MAX_ANSWER_LENGTH)

50
50


In [220]:
train_X = pad_sequences(train_X, maxlen=MAX_QUESTION_LENGTH, padding='post', truncating='post') #truncating dodati ako budem hteo avrage da koristim
train_y = pad_sequences(train_y, maxlen=MAX_ANSWER_LENGTH, padding='post', truncating='post')

In [221]:
len(train_X[0])

50

In [222]:
train_X[0][:50]

array([ 40,  62,   6, 106,  33, 582, 103, 528,  20,   9, 402,  27, 830,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [223]:
train_y_final_output = []
for i in train_y:
    train_y_final_output.append(i[1:]) 

train_y_final_output = pad_sequences(train_y_final_output, MAX_ANSWER_LENGTH, padding='post', truncating='post')

In [224]:
train_y_final_output[0][:50]

array([3751,  123,    5,   18,  621,    1,   88, 5541,   70,   28,  319,
          4,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [225]:
train_y[0][:50]

array([   3, 3751,  123,    5,   18,  621,    1,   88, 5541,   70,   28,
        319,    4,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [226]:
from tensorflow.keras.utils import to_categorical
train_y_final_output = to_categorical(train_y_final_output, MAX_NB_WORDS)

In [227]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input


enc_inp = Input(shape=(50, ))
dec_inp = Input(shape=(50, ))

In [228]:
VOCAB_SIZE = MAX_NB_WORDS
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=50,
                  trainable=True                  
                  )

In [229]:
enc_embed = embed(enc_inp)
enc_lstm = LSTM(200, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]


In [230]:
embed2 = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=50,
                  trainable=True                  
                  )

In [231]:
dec_embed = embed2(dec_inp)
dec_lstm = LSTM(200, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

In [232]:
dense = Dense(VOCAB_SIZE, activation='softmax')
dense_op = dense(dec_op)
model = Model([enc_inp, dec_inp], dense_op)
model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')
print(model.summary())

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 50, 50)       350050      ['input_11[0][0]']               
                                                                                                  
 embedding_5 (Embedding)        (None, 50, 50)       350050      ['input_12[0][0]']               
                                                                                            

In [233]:
model.fit([train_X, train_y],train_y_final_output,epochs=15, validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/15
263/263 [==============================] - 66s 238ms/step - loss: 3.2623 - acc: 0.5454 - val_loss: 2.6192 - val_acc: 0.5576
Epoch 2/15
263/263 [==============================] - 60s 230ms/step - loss: 2.4316 - acc: 0.5873 - val_loss: 2.2876 - val_acc: 0.6086
Epoch 3/15
263/263 [==============================] - 61s 232ms/step - loss: 2.1468 - acc: 0.6282 - val_loss: 2.0584 - val_acc: 0.6423
Epoch 4/15
263/263 [==============================] - 63s 240ms/step - loss: 1.9451 - acc: 0.6580 - val_loss: 1.8999 - val_acc: 0.6716
Epoch 5/15
263/263 [==============================] - 74s 283ms/step - loss: 1.7998 - acc: 0.6838 - val_loss: 1.7775 - val_acc: 0.6898
Epoch 6/15
263/263 [==============================] - 74s 282ms/step - loss: 1.6828 - acc: 0.6992 - val_loss: 1.6880 - val_acc: 0.7009
Epoch 7/15
263/263 [==============================] - 72s 273ms/step - loss: 1.5923 - acc: 0.7099 - val_loss: 1.6200 - val_acc: 0.7083
Epoch 8/15
263/263 [==============================] - 7

In [234]:
enc_model = Model(enc_inp, enc_states)

In [235]:
decoder_state_input_h = Input(shape=(200,))
decoder_state_input_c = Input(shape=(200,))

In [236]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [237]:
decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
#decoder_outputs = dense(decoder_outputs)

In [238]:
dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)

In [266]:
test_data['User'].values[7]

'@AmazonHelp Just did. Thanks'

In [267]:
test_question = tokenizer.texts_to_sequences([test_data['User'].values[7]])

#test_question = tokenizer.texts_to_sequences([train_data['User'].values[0]])
test_question

[[7, 90, 116, 98]]

In [268]:
test_X = pad_sequences(test_question, maxlen=MAX_QUESTION_LENGTH, padding='post', truncating='post')
test_X

array([[  7,  90, 116,  98,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [273]:
test_answer = enc_model.predict( test_X )

In [274]:
empty_target_seq = np.zeros( ( 1 , 1) )
empty_target_seq

array([[0.]])

In [275]:
empty_target_seq[0, 0] = tokenizer.texts_to_sequences(['sos'])[0][0]
empty_target_seq

array([[3.]])

In [276]:
stop_condition = False
decoded_translation=''
i = 0
while not stop_condition :
    dec_outputs , h, c= dec_model.predict([empty_target_seq] + test_answer )
    decoder_concat_input = dense(dec_outputs)
    sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
    
    if i == 0:
        i = 1
        indes = np.argpartition(decoder_concat_input[0, -1, :], -10)[-10:]
        indes = tokenizer.sequences_to_texts([indes])
        print('\n', indes)
        print('\n', sampled_word_index)
    
    sampled_word = tokenizer.sequences_to_texts([[sampled_word_index]])[0] + ' '
    
    if sampled_word != 'eos ':
        decoded_translation += sampled_word  

    if sampled_word == 'eos ' or len(decoded_translation.split()) > 50:
        stop_condition = True 

    empty_target_seq = np.zeros( ( 1 , 1 ) )  
    empty_target_seq[ 0 , 0 ] = sampled_word_index
    ## <SOS> - > hi
    ## hi --> <EOS>
    test_answer = [h, c]

print("Question: ", test_data['User'].values[7])
print("\nExpected: ", test_data['Company'].values[7])
print("\nGiven: ")
print(decoded_translation)


 ["we're we'd oh thanks i we sorry hi please i'm"]

 39
Question:  @AmazonHelp Just did. Thanks

Expected:  <SOS> @135307 Great. Please keep us informed about the outcome.^PJ <EOS>

Given: 
i'm sorry for the trouble with your order please reach out to us here https t co jzp7hla23b so we can look into this with you https t co jzp7hla23b wt 
